## Extracting ClimateTRACE data from raw download files

In [1]:
import os
import pandas as pd

In [2]:
data_path = 'RAW_DATA_PATH'
folders = ['manufacturing/']#, 'mineral_extraction/', 'fossil_fuel_operations/', 'power/', 'waste/']
save_path = "../data\classification_inputs/CT_manufacturing"
key_words = ['asset', 'emissions']

files = []
for folder in folders:
    folder_files = os.listdir(data_path+folder)
    files = files + [data_path+folder+file for file in folder_files if key_words[0] in file and key_words[1] in file and 'transport' not in file]

In [8]:
def filter_df(df:pd.DataFrame, cols:list, filters:list) -> pd.DataFrame:
    """Function for filtering a pandas dataframe based on column values"""
    for col, filt in zip(cols, filters):
        df = df[[i in filt for i in df[col]]]
    return df

In [ ]:
data = pd.DataFrame()
for file in files:
    data = pd.concat((data, pd.read_csv(file)))

data_used = filter_df(data, ['gas'], [['co2e_100yr']])

# Extract lat/longs
data_used['LONGITUDE'] = [i.strip("POINT()").split(" ")[0] for i in data_used['st_astext']]
data_used['LATITUDE'] = [i.strip("POINT()").split(" ")[1] for i in data_used['st_astext']]

# Extract dates
data_used['Year'] = pd.to_datetime(data_used['start_time']).dt.year
data_used['Month'] = pd.to_datetime(data_used['start_time']).dt.month

columns = ['asset_id', 'iso3_country', 'original_inventory_sector', 'capacity', 'asset_name', 'asset_type', 'LONGITUDE', 'LATITUDE', 'Year', 'Month', 'emissions_quantity']

data_used = data_used[columns].rename(columns={'emissions_quantity':'Emissions'}).dropna(subset=['Emissions'])

In [10]:
data_used = data_used.merge(data_used[['Year', 'Month']].drop_duplicates().sort_values(['Year', 'Month']).reset_index(drop=True).reset_index().rename(columns={'index':'Timestep'}), on=['Year', 'Month'])
data_used = data_used.drop(columns=['Year', 'Month'])

In [7]:
feature_cols = [i for i in data_used.columns if i not in ['Timestep', 'Emissions']]
data_pivoted = data_used.pivot(index=feature_cols, columns='Timestep', values = 'Emissions').reset_index()

In [9]:
data_pivoted.to_csv(save_path+'.csv', index=False)
data_used.to_csv(save_path+'_melted.csv', index=False)

In [3]:
data = pd.read_csv(save_path+'.csv')